In [ ]:
#Read Necesssary libraries
import pandas as pd
import numpy as np
import win32com.client as win32
import adodbapi as papi
from pymongo import MongoClient as mc
from datetime import datetime as dt
import os
os.chdir("File Location")
print("Imported Necesssary Files")

# %%

#Source 1
#Connection String for SerialNumber 
Client = mc("MongoDB Connection String")
Collection1 = Client.Dababase_Name.View_Name
cursor1 = Collection1.find()
list1 = list(cursor1)

#Extract data
df = pd.DataFrame(list1)

#Connection String for SerialNumber Return
Collection2 = Client.Database_Name.View_Name
cursor2 = Collection2.find()
list2 = list(cursor2)

df1 = pd.DataFrame(list2)

# %%

# %%

#Source 2
#Extract data from Power BI Semantic model 
conn = papi.connect(
    "Provider=MSOLAP.8; Data Source='powerbi://api.powerbi.com/v1.0/myorg/Workspace Name';initial catalog='Catalog Name';UID ='User Name';PWD ='Password'")
print("connection to service done")

cur1 = conn.cursor()
print("cursor 1 created")
daxmaterial = 'EVALUATE Material'
print("evaluated Material table")
cur1.execute(daxmaterial)
print("cursor 1 executed")
result_material = cur1.fetchall()
print("fetched data for cursor 1")

#Extract Data
Material = pd.DataFrame(data=result_material, index=None,columns=["Brand","Material No","Material","Material Group","Number of Records","Material No(Key)","Material(Key)"])

# %%



#Data Transformation

from datetime import datetime, timedelta
today = datetime.now()
yesterday = today - timedelta(days=1)
start_of_month = datetime(yesterday.year, yesterday.month, 1)
#yesterday=yesterday

#print(start_of_month,yesterday)
##
    
df2 =df._append(df1,ignore_index=True)

df2.loc[df2["transactionType"]=="RT","Quantity"]=-1

df2.loc[(df2["transactionType"]=="GT") | (df2["transactionType"]=="SA"),"Quantity"]=1

df2.drop(df2[df2["transactionType"]=="AB"].index)

df2["Invoice No / Txn Voucher No "]=df2["pos"].apply(str)+"/"+df2["receipt"].apply(str)

df2["Invoice Date/ Txn Date"]=pd.to_datetime(df2["businessDate"])
#print(df2["Invoice Date/ Txn Date"])

df2 = df2[(df2["Invoice Date/ Txn Date"]>=start_of_month) & (df2["Invoice Date/ Txn Date"]<=yesterday) ]
df2["Invoice Date/ Txn Date"] = df2["Invoice Date/ Txn Date"].dt.strftime('%d-%m-%Y')
#print(df2["Invoice Date/ Txn Date"])


Material["Material(Key)"]=Material["Material(Key)"].astype('int64')

df3=pd.merge(df2,Material,left_on="itemID",right_on="Material(Key)",how="left")

Result = df3[["Invoice Date/ Txn Date","Invoice No / Txn Voucher No ","retailStoreId","Material No(Key)","itemDesc","serialno","Quantity","Material(Key)","brand"]]

Result=Result.rename(columns={'retailStoreId':'Outlet ID','Material No(Key)':'Part No (SKU)','itemDesc':'Product Description','Material(Key)':'Article'})

Result1 = Result.dropna(subset=["Part No (SKU)","Quantity"]).reset_index(drop=True)
Result1['serialno']=Result1['serialno'].replace("'", '',regex=True)
Result1 = Result1.drop_duplicates().reset_index(drop=True)

#%%

#%%
writer = pd.ExcelWriter(f'SERIAL NO REPORT_01-{yesterday.strftime("%m")}-{yesterday.strftime("%Y")} to {yesterday.strftime("%d")}-{yesterday.strftime("%m")}-{yesterday.strftime("%Y")}.xlsx', engine='xlsxwriter')
Result1.to_excel(writer, sheet_name='Summary', )
writer._save()

#%%

#Send mail to the specified mail id

outlook = win32.Dispatch('outlook.application')
mail = outlook.CreateItem(0)
mail.Subject = f'SERIAL NO REPORT_01-{yesterday.strftime("%m")}-{yesterday.strftime("%Y")} to {yesterday.strftime("%d")}-{yesterday.strftime("%m")}-{yesterday.strftime("%Y")}'
mail.Attachments.Add(os.getcwd()+rf"\\SERIAL NO REPORT_01-{yesterday.strftime('%m')}-{yesterday.strftime('%Y')} to {yesterday.strftime('%d')}-{yesterday.strftime('%m')}-{yesterday.strftime('%Y')}.xlsx")
mail.To = 'mail id'
mail.CC = 'list mail ids'
mail.HTMLBody = rf'''Hi,
<p style="color: white;">Auto forward id: 05092023serial</p>
Please find the report for the date from 01-{yesterday.strftime('%m')}-{yesterday.strftime('%Y')} to {yesterday.strftime('%d')}-{yesterday.strftime('%m')}-{yesterday.strftime('%Y')}
'''
mail.send